**1. Read data**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import torch
import json

# Đọc dữ liệu từ tệp tin JSON
with open('zac2022_train_merged_final.json', 'rb') as f:
    data = json.load(f)

data = data["data"]

# New Section

In [3]:
full_data = []
dict = {}
for item in data:
  if item['category'] == "FULL_ANNOTATION":
    full_data.append(item)


print(len(full_data))
#print(full_data)

4989


In [4]:
for obj in full_data:
    del obj["short_candidate_start"]
    del obj["answer"]
    del obj["category"]
    del obj["is_long_answer"]
    del obj["title"]
    #del obj["id"]
print(full_data[2])

{'id': 'ee8a272bcefac7b96037a1571984b1f8', 'question': 'Thành phố nào là thủ phủ của Ai Cập trong đế quốc La Mã', 'text': 'Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , và Alexandria trở thành thủ phủ của tỉnh Ai Cập trong đế quốc La Mã .', 'short_candidate': 'Alexandria'}


a. Tìm vị trí đầu và cuối của answer(short_candidate) trong text

In [5]:
def get_start_end_idx(story):
    str_idx = story['text'].find(story['short_candidate'])
    end_idx = str_idx + len(story['short_candidate'])
    return {'str_idx':str_idx,
          'end_idx': end_idx}

In [6]:
for item in full_data:
  item['str_idx'] = get_start_end_idx(item)['str_idx']
  item['end_idx'] = get_start_end_idx(item)['end_idx']

print(full_data[0]['str_idx'])

53


In [7]:
!pip install datasets
from datasets import DatasetDict



In [8]:
data_train = full_data
train_ds = data_train[:4000]
test_ds = data_train[4000:]

In [9]:
qa_dataset = DatasetDict()
qa_dataset.update({"train": train_ds})
qa_dataset.update({"test": test_ds})

In [10]:
import pandas as pd
from datasets import Dataset

# Your list of dictionaries

# Create a pandas DataFrame from your list
my_dataframe1 = pd.DataFrame(train_ds)
my_dataframe2 = pd.DataFrame(test_ds)

# Create a dataset from your dataframe
train_ds = Dataset.from_pandas(my_dataframe1)
test_ds = Dataset.from_pandas(my_dataframe2)
print(type(test_ds))

<class 'datasets.arrow_dataset.Dataset'>


In [11]:
print("text: ", train_ds[0]["text"])
print("question: ", train_ds[0]["question"])
print(train_ds[0])


text:  có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...
question:  Đất nước nào không có quân đội
{'id': 'c926e7b0717202618a10dd907d4b4c39', 'question': 'Đất nước nào không có quân đội', 'text': 'có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...', 'short_candidate': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...', 'str_idx': 53, 'end_idx': 126}


**2. Tokenize**

In [12]:
pip install git+https://github.com/datquocnguyen/transformers

  Cloning https://github.com/datquocnguyen/transformers to /tmp/pip-req-build-ada7nkus
  Running command git clone --filter=blob:none --quiet https://github.com/datquocnguyen/transformers /tmp/pip-req-build-ada7nkus
  Resolved https://github.com/datquocnguyen/transformers to commit 83cb6dab92114f8772a0f3d955b58bcb039dfb06
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
from transformers import BartphoTokenizerFast

tokenizer = BartphoTokenizerFast.from_pretrained("vinai/bartpho-word-base")
#tokenizer.is_fast

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BartphoTokenizerFast'.


In [14]:
text = train_ds[0]["text"]
question = train_ds[0]["question"]

inputs = tokenizer(question,text)
tokenizer.decode(inputs["input_ids"])

'<s>Đất nước nào không có quân đội </s></s>có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican... </s>'

In [15]:
inputs = tokenizer(
    question,
    text,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    print()

<s>Đất nước nào không có quân đội </s></s>có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican... </s>



In [16]:
inputs = tokenizer(
    question,
    text,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [17]:
inputs["overflow_to_sample_mapping"]

[0]

In [18]:
inputs = tokenizer(
    train_ds[2:5]["question"],
    train_ds[2:5]["text"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 5 features.
Here is where each comes from: [0, 1, 2, 2, 2].


In [19]:
train_ds[2:5]

{'id': ['ee8a272bcefac7b96037a1571984b1f8',
  'c51604b25a641b2665c4ce1784b8cc6a',
  '04bf774a424133d27c34895960937b87'],
 'question': ['Thành phố nào là thủ phủ của Ai Cập trong đế quốc La Mã',
  'Ai là người đứng đầu trong cuộc chống lại chính quyền Đông Hán',
  'Nền văn hoá Phục Hưng bắt nguồn từ nước nào'],
 'text': ['Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , và Alexandria trở thành thủ phủ của tỉnh Ai Cập trong đế quốc La Mã .',
  'Các nghiên cứu lịch sử cho thấy Cổ Am rất có thể là nơi nữ tướng Lê Chân (? - 43) dựng căn cứ để luyện tập nghĩa quân trong cuộc khởi nghĩa của Hai Bà Trưng (40) chống lại ách đô hộ của nhà Hán.',
  'Phục Hưng (tiếng Pháp: "Renaissance", , , từ "ri-" "lần nữa" và "nascere" "được sinh ra") là một phong trào văn hóa thường được xem là bao gồm giai đoạn từ thế kỷ XV đến thế kỷ XVII, khởi đầu tại Firenze (Ý) vào Hậu kỳ Trung Đại, sau đó lan rộng ra phần còn lại của châu Âu trên những quy mô và mức độ khác nhau. Người ta cũng dùng từ Phục Hưng để chỉ, một 

In [20]:
answers = train_ds[2:5]["short_candidate"]
idxs = train_ds[2:5]["str_idx"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = idxs[sample_idx]
    end_char = start_char + len(answer)
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([33, 32, 84, 48, 0], [33, 33, 84, 48, 0])

In [21]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Alexandria, labels give: Alexandria


In [22]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: Ý, decoded example: <s>Nền văn hoá Phục Hưng bắt nguồn từ nước nào </s></s>) vào Hậu kỳ Trung Đại, sau đó lan rộng ra phần còn lại của châu Âu trên những quy mô và mức độ khác nhau. Người ta cũng dùng từ Phục Hưng để chỉ, một cách không nhất quán, thời kỳ lịch sử diễn ra phong trào văn hóa nói trên. </s>


In [23]:
max_length = 1024
stride = 128

def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
         max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["short_candidate"]
    idxs = examples["str_idx"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = idxs[sample_idx]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [24]:
train_dataset = train_ds.map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=train_ds.column_names,
)
len(train_ds), len(train_dataset)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

(4000, 4000)

In [25]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [26]:
test_dataset = test_ds.map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=test_ds.column_names,
)
len(test_ds), len(test_dataset)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

(989, 989)

In [ ]:
!pip install evaluate

In [28]:
!pip install evaluate

In [29]:
#!pip install TensorFlow==2.14.0

In [30]:
from tqdm.auto import tqdm
import evaluate
import numpy as np
import collections

n_best = 20
max_answer_length = 30
predicted_answers = []
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [31]:
!pip install transformers[torch]
!pip install accelerate -U


In [32]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-word-base")

epochs = 10
batch_size = 4
lr = 2e-5

training_args = TrainingArguments(
    output_dir='results',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=6,  # batch size per device during training
    per_device_eval_batch_size=6,   # batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,            # directory for storing logs
    logging_steps=200,
    # label_names=['input_ids','attention_mask'] ,
    eval_steps = 200,
    # eval_accumulation_steps=1,
    # report_to='tensorboard',
    # save_total_limit=1,
    evaluation_strategy="steps",
    save_steps=200,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-word-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
200,2.997500,No log
400,2.073000,No log
600,1.674700,No log
800,1.233300,No log


Step,Training Loss,Validation Loss
200,2.997500,No log
400,2.073000,No log
600,1.674700,No log
800,1.233300,No log
1000,1.051700,No log
1200,0.937400,No log
1400,0.752500,No log
1600,0.534000,No log
1800,0.484800,No log
2000,0.503900,No log


TrainOutput(global_step=2001, training_loss=1.2237846064216074, metrics={'train_runtime': 4343.7014, 'train_samples_per_second': 2.763, 'train_steps_per_second': 0.461, 'total_flos': 7317177384960000.0, 'train_loss': 1.2237846064216074, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
from transformers import pipeline,AutoTokenizer,AutoModel,AutoConfig

# Replace this with your own checkpoint
model = AutoModel.from_pretrained("results/checkpoint-1000",local_files_only=True)

question_answerer = pipeline("question-answering",model=model)

context = "Các nghiên cứu lịch sử cho thấy Cổ Am rất có thể là nơi nữ tướng Lê Chân (? - 43) dựng căn cứ để luyện tập nghĩa quân trong cuộc khởi nghĩa của Hai Bà Trưng (40) chống lại ách đô hộ của nhà Hán."

question = "Nền văn hoá Phục Hưng bắt nguồn từ nước nào"
question_answerer(question=question, context=context)